In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('dataset.csv')
print(df.head(10))

            Disease   Symptom_1  ... Symptom_16 Symptom_17
0  Fungal infection     itching  ...        NaN        NaN
1  Fungal infection   skin_rash  ...        NaN        NaN
2  Fungal infection     itching  ...        NaN        NaN
3  Fungal infection     itching  ...        NaN        NaN
4  Fungal infection     itching  ...        NaN        NaN
5  Fungal infection   skin_rash  ...        NaN        NaN
6  Fungal infection     itching  ...        NaN        NaN
7  Fungal infection     itching  ...        NaN        NaN
8  Fungal infection     itching  ...        NaN        NaN
9  Fungal infection     itching  ...        NaN        NaN

[10 rows x 18 columns]


**Mapping the diseases to an integer value**

In [3]:
df1 = pd.read_csv('dataset.csv',index_col='Disease')
df.replace({'Disease':{'Fungal infection':0,'Allergy':1,'GERD':2,'Chronic cholestasis':3,'Drug Reaction':4,
            'Peptic ulcer diseae':5,'AIDS':6,'Diabetes ':7,'Gastroenteritis':8,'Bronchial Asthma':9,'Hypertension ':10,
            'Migraine':11,'Cervical spondylosis':12,
            'Paralysis (brain hemorrhage)':13,'Jaundice':14,'Malaria':15,'Chicken pox':16,'Dengue':17,'Typhoid':18,'hepatitis A':19,
            'Hepatitis B':20,'Hepatitis C':21,'Hepatitis D':22,'Hepatitis E':23,'Alcoholic hepatitis':24,'Tuberculosis':25,
            'Common Cold':26,'Pneumonia':27,'Dimorphic hemmorhoids(piles)':28,'Heart attack':29,'Varicose veins':30,'Hypothyroidism':31,
            'Hyperthyroidism':32,'Hypoglycemia':33,'Osteoarthristis':34,'Arthritis':35,
            '(vertigo) Paroymsal  Positional Vertigo':36,'Acne':37,'Urinary tract infection':38,'Psoriasis':39,'Impetigo':40}},inplace=True)

df.head(15)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,0,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Performing one hot encoding using pandas Pivot table:**

In [4]:
cols = [i for i in df.iloc[:, 1:].columns]
temp = pd.melt(df.reset_index(), id_vars=['index'], value_vars=cols)
temp['add1'] = 1
diseases = pd.pivot_table(temp, values = 'add1', index = 'index', columns = 'value')
diseases.insert(0, 'Disease', df['Disease'])
diseases = diseases.fillna(0)
diseases.head(15)

value,Disease,abdominal_pain,abnormal_menstruation,acidity,acute_liver_failure,altered_sensorium,anxiety,back_pain,belly_pain,blackheads,bladder_discomfort,blister,blood_in_sputum,bloody_stool,blurred_and_distorted_vision,breathlessness,brittle_nails,bruising,burning_micturition,chest_pain,chills,cold_hands_and_feets,coma,congestion,constipation,continuous_feel_of_urine,continuous_sneezing,cough,cramps,dark_urine,dehydration,depression,diarrhoea,dischromic _patches,distention_of_abdomen,dizziness,drying_and_tingling_lips,enlarged_thyroid,excessive_hunger,extra_marital_contacts,...,restlessness,runny_nose,rusty_sputum,scurring,shivering,silver_like_dusting,sinus_pressure,skin_peeling,skin_rash,slurred_speech,small_dents_in_nails,spinning_movements,spotting_ urination,stiff_neck,stomach_bleeding,stomach_pain,sunken_eyes,sweating,swelled_lymph_nodes,swelling_joints,swelling_of_stomach,swollen_blood_vessels,swollen_extremeties,swollen_legs,throat_irritation,toxic_look_(typhos),ulcers_on_tongue,unsteadiness,visual_disturbances,vomiting,watering_from_eyes,weakness_in_limbs,weakness_of_one_body_side,weight_gain,weight_loss,yellow_crust_ooze,yellow_urine,yellowing_of_eyes,yellowish_skin,itching
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [7]:
print('Total Diseases: ',len(diseases['Disease'].unique()))

Total Diseases:  41


In [8]:
df2 = diseases.drop('Disease',axis=1)
l1=df2.columns
l2 = []
for i in range(0,len(l1)):
  l2.append(0)
len(l2)

131

In [9]:
y = diseases.Disease
X = diseases.drop('Disease', axis = 1)
print(X)

value   abdominal_pain   abnormal_menstruation  ...   yellowish_skin  itching
index                                           ...                          
0                  0.0                     0.0  ...              0.0      1.0
1                  0.0                     0.0  ...              0.0      0.0
2                  0.0                     0.0  ...              0.0      1.0
3                  0.0                     0.0  ...              0.0      1.0
4                  0.0                     0.0  ...              0.0      1.0
...                ...                     ...  ...              ...      ...
4915               0.0                     0.0  ...              0.0      0.0
4916               0.0                     0.0  ...              0.0      0.0
4917               0.0                     0.0  ...              0.0      0.0
4918               0.0                     0.0  ...              0.0      0.0
4919               0.0                     0.0  ...             

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25 ,shuffle = True, random_state = 31)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape) 

(3690, 131)
(3690,)
(1230, 131)


**Building a simple `Artificial Neural Network (ANN)` in tensorflow**

In [13]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(131, )))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(41, activation='softmax'))

In [12]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               16896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 41)                10537     
Total params: 60,457
Trainable params: 60,457
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
model.compile(optimizer= tf.keras.optimizers.Adam(lr = 0.0005) , loss='sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=7)

Epoch 1/7
116/116 [==============================] - 1s 2ms/step - loss: 3.3452 - sparse_categorical_accuracy: 0.3072
Epoch 2/7
116/116 [==============================] - 0s 2ms/step - loss: 0.9903 - sparse_categorical_accuracy: 0.9533
Epoch 3/7
116/116 [==============================] - 0s 2ms/step - loss: 0.1753 - sparse_categorical_accuracy: 0.9903
Epoch 4/7
116/116 [==============================] - 0s 2ms/step - loss: 0.0740 - sparse_categorical_accuracy: 0.9961
Epoch 5/7
116/116 [==============================] - 0s 2ms/step - loss: 0.0429 - sparse_categorical_accuracy: 0.9977
Epoch 6/7
116/116 [==============================] - 0s 2ms/step - loss: 0.0296 - sparse_categorical_accuracy: 0.9975
Epoch 7/7
116/116 [==============================] - 0s 2ms/step - loss: 0.0203 - sparse_categorical_accuracy: 0.9996


In [16]:
test_loss, accuracy = model.evaluate(X_test, y_test)

39/39 [==============================] - 0s 1ms/step - loss: 0.0022 - sparse_categorical_accuracy: 1.0000


In [17]:
Res = ['Fungal infection','Allergy','GERD','Chronic cholestasis','Drug Reaction',
    'Peptic ulcer diseae','AIDS','Diabetes ','Gastroenteritis','Bronchial Asthma','Hypertension ',
    'Migraine','Cervical spondylosis',
    'Paralysis (brain hemorrhage)','Jaundice','Malaria','Chicken pox','Dengue','Typhoid','hepatitis A',
    'Hepatitis B','Hepatitis C','Hepatitis D','Hepatitis E','Alcoholic hepatitis','Tuberculosis',
    'Common Cold','Pneumonia','Dimorphic hemmorhoids(piles)','Heart attack','Varicose veins','Hypothyroidism',
    'Hyperthyroidism','Hypoglycemia','Osteoarthristis','Arthritis',
    '(vertigo) Paroymsal  Positional Vertigo','Acne','Urinary tract infection','Psoriasis',
    'Impetigo']

In [31]:
#printing all the Symptoms
symptoms = list(df2.columns)
print('Symptoms: ')
num = 0
for i in symptoms:
  print(num,':', i)
  num+=1

Symptoms: 
0 :  abdominal_pain
1 :  abnormal_menstruation
2 :  acidity
3 :  acute_liver_failure
4 :  altered_sensorium
5 :  anxiety
6 :  back_pain
7 :  belly_pain
8 :  blackheads
9 :  bladder_discomfort
10 :  blister
11 :  blood_in_sputum
12 :  bloody_stool
13 :  blurred_and_distorted_vision
14 :  breathlessness
15 :  brittle_nails
16 :  bruising
17 :  burning_micturition
18 :  chest_pain
19 :  chills
20 :  cold_hands_and_feets
21 :  coma
22 :  congestion
23 :  constipation
24 :  continuous_feel_of_urine
25 :  continuous_sneezing
26 :  cough
27 :  cramps
28 :  dark_urine
29 :  dehydration
30 :  depression
31 :  diarrhoea
32 :  dischromic _patches
33 :  distention_of_abdomen
34 :  dizziness
35 :  drying_and_tingling_lips
36 :  enlarged_thyroid
37 :  excessive_hunger
38 :  extra_marital_contacts
39 :  family_history
40 :  fast_heart_rate
41 :  fatigue
42 :  fluid_overload
43 :  foul_smell_of urine
44 :  headache
45 :  high_fever
46 :  hip_joint_pain
47 :  history_of_alcohol_consumption
4

In [20]:
print(diseases)

value  Disease   abdominal_pain  ...   yellowish_skin  itching
index                            ...                          
0            0              0.0  ...              0.0      1.0
1            0              0.0  ...              0.0      0.0
2            0              0.0  ...              0.0      1.0
3            0              0.0  ...              0.0      1.0
4            0              0.0  ...              0.0      1.0
...        ...              ...  ...              ...      ...
4915        36              0.0  ...              0.0      0.0
4916        37              0.0  ...              0.0      0.0
4917        38              0.0  ...              0.0      0.0
4918        39              0.0  ...              0.0      0.0
4919        40              0.0  ...              0.0      0.0

[4920 rows x 132 columns]


In [21]:
#Testing the model
abc = [' cold_hands_and_feets', ' continuous_sneezing', ' cough']
for k in range(0,len(l1)):
  for z in abc:
    if(z==l1[k]):
      l2[k] = 1
iptest = [l2]
sum = 0
for i in l2:
  if i == 1:
    sum+=i
print('Length of l2: ',sum)
print(l2)         
#pred = np.argmax(model.predict(iptest), axis = -1)
pred = model.predict(iptest)

Length of l2:  3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
new = list(pred[0])
keys = new
values = Res
pairs = dict(zip(keys,values))
print(pairs)

{0.018800644: 'Fungal infection', 0.46861634: 'Allergy', 0.04720085: 'GERD', 0.009136137: 'Chronic cholestasis', 0.006515919: 'Drug Reaction', 0.024270479: 'Peptic ulcer diseae', 0.020882355: 'AIDS', 0.0025102594: 'Diabetes ', 0.005662171: 'Gastroenteritis', 0.02408057: 'Bronchial Asthma', 0.015818872: 'Hypertension ', 0.018310009: 'Migraine', 0.015259683: 'Cervical spondylosis', 0.008779914: 'Paralysis (brain hemorrhage)', 0.0034757485: 'Jaundice', 0.007188558: 'Malaria', 0.008750871: 'Chicken pox', 0.015065896: 'Dengue', 0.008106595: 'Typhoid', 0.0057654562: 'hepatitis A', 0.0026433454: 'Hepatitis B', 0.0023072076: 'Hepatitis C', 0.0037221462: 'Hepatitis D', 0.00298488: 'Hepatitis E', 0.010361937: 'Alcoholic hepatitis', 0.0143026225: 'Tuberculosis', 0.031837795: 'Common Cold', 0.016571335: 'Pneumonia', 0.020683086: 'Dimorphic hemmorhoids(piles)', 0.01043784: 'Heart attack', 0.0026830328: 'Varicose veins', 0.044690702: 'Hypothyroidism', 0.003856288: 'Hyperthyroidism', 0.0063672247: 'H

In [25]:
pairs1 = dict(sorted(pairs.items(), reverse = True))
prob = []
disease = []
for k,v in pairs1.items():
  prob.append(k)
  disease.append(v) 

In [26]:
#Printing 3 most probable disease along with their probability
import itertools
final = dict(itertools.islice(pairs1.items(), 3))
print(final)

{0.46861634: 'Allergy', 0.04720085: 'GERD', 0.044690702: 'Hypothyroidism'}


In [64]:
model_json = model.to_json()
with open("symptom_predictor.json", "w") as json_file:
    json_file.write(model_json)

In [27]:
model.save_weights("symptom_predictor.h5")